In [1]:
import pandas as pd
import pymer4.models
import numpy as np
import scipy.stats
import os

# Output and formatting settings

In [2]:
output_path = "C:\\Users\\azgonnikov\\Dropbox\\Apps\\Overleaf\\AAP Dynamics of merging decisions Elsevier template"

column_names = {"Estimate": "$\\beta$", "Z-stat": "$z$", "P-val": "$p$", "T-stat": "$t$", "F-stat": "$F$", "2.5_ci": "CI 2.5\%", "97.5_ci": "CI 97.5\%", "NumDF": "df"}
var_names = {"tta_z": "$\\textrm{TTA}$",
             "d_z": "distance",
             "time_budget_z": "time budget",
             "decision1": "decision",
             "dwell_mirror_z": "\% dwell time mirror",
             "RT_z": "RT",
             "tta_z:time_budget_z": "$\\textrm{TTA}$:time budget",
             "d_z:dwell_mirror_z": "distance:\% dwell time mirror",
             "tta_z:dwell_mirror_z": "$\\textrm{TTA}$:\% dwell time mirror",
             "time_budget_z:dwell_mirror_z": "time budget:\% dwell time mirror",
             "tta_z:time_budget_z:dwell_mirror_z": "$\\textrm{TTA}$:time budget:\% dwell time mirror",
             "decision1:d_z": "decision:distance",
             "decision1:tta_z": "decision:$\\textrm{TTA}$",
             "decision1:time_budget_z": "decision:time budget",
             "decision1:tta_z:time_budget_z": "decision:$\\textrm{TTA}$:time budget"}

def p_formatted(p):
    if p>0.01:
        return "{:.2f}".format(p)
    elif p>0.001:
        return "{:.3f}".format(p)
    else:
        return "$<0.001$"

# Read and pre-process the data

In [3]:
metrics = pd.read_csv("metrics.csv")

def get_z_score(x):
    return (x-x.mean())/x.std()

for col in ["RT", "d", "tta", "time_budget", "dwell_mirror"]:
    metrics.loc[:, col+"_z"] = get_z_score(metrics[col]) 

# Decision outcome as a function of kinematic conditions

In [4]:
model_decision = pymer4.models.Lmer("is_gap_accepted ~ 1 + d_z + tta_z + time_budget_z + (1 + tta_z + time_budget_z + d_z | participant)", data=metrics, family="binomial")
model_decision_fit = model_decision.fit(summarize=True)
model_decision.coefs

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: is_gap_accepted~1+d_z+tta_z+time_budget_z+(1+tta_z+time_budget_z+d_z|participant)

Family: binomial	 Inference: parametric

Number of observations: 8995	 Groups: {'participant': 25.0}

Log-likelihood: -3322.419 	 AIC: 6672.839

Random effects:

                      Name    Var    Std
participant    (Intercept)  1.690  1.300
participant          tta_z  0.278  0.528
participant  time_budget_z  0.210  0.459
participant            d_z  0.269  0.518

                       IV1            IV2   Corr
participant    (Intercept)          tta_z -0.183
participant    (Intercept)  time_budget_z -0.195
participant    (Intercept)            d_z  0.286
participant          tta_z  time_budget_z -0.215
participant          tta_z            d_z -0.594
participant  time_budget_z            d_z  0.451

Fixed effects:


,Estimate,2.5_ci,97.5_ci,SE,OR,OR_2.5_ci,OR_97.5_ci,Prob,Prob_2.5_ci,Prob_97.5_ci,Z-stat,P-val,Sig
(Intercept),1.258766,0.739833,1.777700,0.264767,3.521075,2.095585,5.916231,0.778814,0.676959,0.855413,4.754245,1.991893e-06,***
d_z,0.492549,0.278481,0.706616,0.109220,1.636482,1.321122,2.027121,0.620707,0.569174,0.669653,4.509684,6.492412e-06,***
tta_z,1.878709,1.650087,2.107330,0.116646,6.545048,5.207435,8.226248,0.867463,0.838903,0.891614,16.106116,2.310912e-58,***
time_budget_z,-0.480137,-0.671919,-0.288355,0.097850,0.618698,0.510727,0.749495,0.382220,0.338067,0.428407,-4.906878,9.253765e-07,***


In [5]:
coefs = model_decision.coefs.loc[:, ["Estimate", "SE", "Z-stat", "P-val"]]
coefs["P-val"] = coefs["P-val"].apply(p_formatted)
styler = coefs.rename(columns=column_names, index=var_names).style.format(precision=2)

with open(os.path.join(output_path, "tab_decision.tex"), 'w') as writer:
     writer.write(styler.to_latex(
         column_format="rrrrr", position="h", position_float="centering",
         hrules=True, label="tab:decision", caption="Standardized coefficients of the mixed-effects logistic regression describing the final decision. All effects were modelled as random slopes per participant: \\texttt{decision $\sim$ 1 + distance + TTA + time budget + (1 + distance + TTA + time budget) | participant}."
     )
)

# Response time

In [6]:
model_RT = pymer4.models.Lmer("RT_z ~ 1 + decision*(d_z + tta_z*time_budget_z) + (decision | participant) ", data=metrics, family="gaussian")
model_RT.fit(summarize=True, factors={"decision": ["Accept", "Reject"]})
model_RT.coefs

Linear mixed model fit by REML [’lmerMod’]
Formula: RT_z~1+decision*(d_z+tta_z*time_budget_z)+(decision|participant)

Family: gaussian	 Inference: parametric

Number of observations: 8995	 Groups: {'participant': 25.0}

Log-likelihood: -9553.247 	 AIC: 19134.494

Random effects:

                       Name    Var    Std
participant     (Intercept)  0.346  0.588
participant  decisionReject  0.212  0.461
Residual                     0.475  0.689

                     IV1             IV2   Corr
participant  (Intercept)  decisionReject -0.413

Fixed effects:


,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val,Sig
(Intercept),-0.254158,-0.485713,-0.022603,0.118143,24.096480,-2.151284,4.168976e-02,*
decision1,1.041423,0.855107,1.227738,0.095061,24.967676,10.955331,5.007392e-11,***
d_z,-0.032367,-0.050265,-0.014469,0.009132,8938.891993,-3.544493,3.953779e-04,***
tta_z,0.044018,0.023821,0.064216,0.010305,8950.164891,4.271539,1.961353e-05,***
time_budget_z,0.066641,0.047053,0.086229,0.009994,8939.305508,6.668119,2.744551e-11,***
tta_z:time_budget_z,0.044605,0.025049,0.064160,0.009977,8938.454995,4.470559,7.897711e-06,***
decision1:d_z,0.079340,0.048490,0.110189,0.015740,8950.607096,5.040672,4.729855e-07,***
decision1:tta_z,0.266949,0.224476,0.309422,0.021670,8958.597597,12.318615,1.369146e-34,***
decision1:time_budget_z,0.125020,0.083797,0.166243,0.021032,8941.990641,5.944157,2.882877e-09,***
decision1:tta_z:time_budget_z,0.041695,0.000679,0.082710,0.020927,8940.365160,1.992413,4.635617e-02,*


In [7]:
coefs = model_RT.coefs.loc[:, ["Estimate", "SE", "T-stat", "P-val"]]
coefs["P-val"] = coefs["P-val"].apply(p_formatted)
styler = coefs.rename(columns=column_names, index=var_names).style.format(precision=2)

with open(os.path.join(output_path, "tab_RT.tex"), 'w') as writer:
     writer.write(styler.to_latex(
         column_format="rrrrr", position="h", position_float="centering",
         hrules=True, label="tab:RT", caption="Standardized coefficients of the mixed-effects linear regression describing response times. Random slope of decision was included per participant: \\texttt{RT $\sim$ 1 + decision*(TTA*time budget + distance) + (1 + decision) | participant}. ``Accept'' was set as a reference level for the decision outcome factor."
     )
)

## ANOVA

In [8]:
RT_anova = model_RT.anova()
RT_decision_marginal_estimates, RT_decision_comparisons = model_RT.post_hoc(marginal_vars=["decision"])
RT_anova

SS Type III Analysis of Variance Table with Satterthwaite approximated degrees of freedom:
(NOTE: Using original model contrasts, orthogonality not guaranteed)


,SS,MS,NumDF,DenomDF,F-stat,P-val,Sig
decision,57.027620,57.027620,1,24.967676,120.019279,5.007392e-11,***
d_z,0.409220,0.409220,1,8953.722085,0.861237,3.534179e-01,
tta_z,127.461269,127.461269,1,8956.022842,268.252639,1.983653e-59,***
time_budget_z,71.681515,71.681515,1,8945.396627,150.859595,2.127449e-34,***
tta_z:time_budget_z,18.593343,18.593343,1,8941.111742,39.131208,4.144361e-10,***
decision:d_z,12.072885,12.072885,1,8950.607096,25.408371,4.729855e-07,***
decision:tta_z,72.103772,72.103772,1,8958.597597,151.748269,1.369146e-34,***
decision:time_budget_z,16.788614,16.788614,1,8941.990641,35.333008,2.882877e-09,***
decision:tta_z:time_budget_z,1.886222,1.886222,1,8940.365160,3.969708,4.635617e-02,*


In [9]:
coefs = RT_anova.loc[:, ["SS", "MS", "F-stat", "P-val"]]
coefs["P-val"] = coefs["P-val"].apply(p_formatted)
styler = coefs.rename(columns=column_names, index=var_names).style.format(precision=2)

with open(os.path.join(output_path, "tab_RT_ANOVA.tex"), 'w') as writer:
     writer.write(styler.to_latex(
         column_format="rrrrrr", position="h", position_float="centering",
         hrules=True, label="tab:RT_ANOVA", caption="ANOVA table based on the mixed-effects linear regression describing response time. Random slope of decision was included per participant: \\texttt{RT $\sim$ 1 + decision*(TTA*time budget + distance) + (1 + decision) | participant}."
     )
)

## Difference between accept and reject RTs

In [10]:
RT_decision_marginal_estimates, RT_decision_comparisons = model_RT.post_hoc(marginal_vars=["decision"])
RT_decision_comparisons

,Contrast,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val,Sig
1,Accept - Reject,-1.041,-1.237,-0.846,0.095,24.967,-10.956,0.0,***


In [11]:
RT_decision_comparisons.Estimate*metrics.RT.std()

1   -0.645955
Name: Estimate, dtype: float64

## Estimates of condition effects on RT per decision

In [12]:
def get_marginal_estimates(model_RT, marginal_vars):
    marginal_estimates, comparisons = model_RT.post_hoc(marginal_vars=marginal_vars, grouping_vars=["decision"])
    marginal_estimates["T-stat"] = marginal_estimates["Estimate"]/marginal_estimates["SE"]
    marginal_estimates["P-val"] = scipy.stats.t.sf(np.abs(marginal_estimates["T-stat"]), marginal_estimates.DF)
    return marginal_estimates

In [13]:
get_marginal_estimates(model_RT, "tta_z")

,decision,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val
1,Accept,0.044,0.024,0.064,0.010,8950.164,4.400000,5.475230e-06
2,Reject,0.311,0.274,0.348,0.019,8957.837,16.368421,1.162934e-59


In [14]:
get_marginal_estimates(model_RT, "d_z")

,decision,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val
1,Accept,-0.032,-0.050,-0.014,0.009,8938.892,-3.555556,0.000190
2,Reject,0.047,0.022,0.072,0.013,8956.576,3.615385,0.000151


In [15]:
get_marginal_estimates(model_RT, "time_budget_z")

,decision,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val
1,Accept,0.067,0.047,0.086,0.010,8939.306,6.700000,1.105043e-11
2,Reject,0.192,0.155,0.228,0.019,8944.907,10.105263,3.515585e-24


## Estimates of time budget effect per decision and TTA level

In [16]:
marginal_estimates, comparisons = model_RT.post_hoc(marginal_vars=["time_budget_z"], grouping_vars=["decision", "tta_z"])
marginal_estimates["T-stat"] = marginal_estimates["Estimate"]/marginal_estimates["SE"]
marginal_estimates["P-val"] = scipy.stats.t.sf(np.abs(marginal_estimates["T-stat"]), marginal_estimates.DF)

In [17]:
marginal_estimates

,decision,tta_z,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val
1,Accept,-1.0,0.022,-0.011,0.055,0.017,8939.527,1.294118,9.782911e-02
2,Reject,-1.0,0.105,0.079,0.132,0.014,8949.606,7.500000,3.495322e-14
3,Accept,1.0,0.111,0.090,0.132,0.011,8937.299,10.090909,4.063917e-24
4,Reject,1.0,0.278,0.211,0.345,0.034,8942.048,8.176471,1.661016e-16


# Decision outcome as a function of dwell time and response time

In [18]:
model_decision_dwell_RT = pymer4.models.Lmer("is_gap_accepted ~ (d_z + tta_z + time_budget_z)*dwell_mirror_z + RT_z + (tta_z + time_budget_z | participant) ", data=metrics, family="binomial")
model_decision_dwell_RT_fit = model_decision_dwell_RT.fit(summarize=True)
model_decision_dwell_RT.coefs

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: is_gap_accepted~(d_z+tta_z+time_budget_z)*dwell_mirror_z+RT_z+(tta_z+time_budget_z|participant)

Family: binomial	 Inference: parametric

Number of observations: 8995	 Groups: {'participant': 25.0}

Log-likelihood: -2609.598 	 AIC: 5249.196

Random effects:

                      Name    Var    Std
participant    (Intercept)  2.753  1.659
participant          tta_z  0.648  0.805
participant  time_budget_z  0.126  0.355

                     IV1            IV2   Corr
participant  (Intercept)          tta_z  0.392
participant  (Intercept)  time_budget_z -0.525
participant        tta_z  time_budget_z -0.320

Fixed effects:


,Estimate,2.5_ci,97.5_ci,SE,OR,OR_2.5_ci,OR_97.5_ci,Prob,Prob_2.5_ci,Prob_97.5_ci,Z-stat,P-val,Sig
(Intercept),1.650792,0.988030,2.313555,0.338150,5.211107,2.685938,10.110299,0.838998,0.728699,0.909993,4.881830,1.051056e-06,***
d_z,0.469337,0.397102,0.541573,0.036856,1.598934,1.487507,1.718709,0.615227,0.597991,0.632178,12.734472,3.804091e-37,***
tta_z,2.242300,1.901045,2.583554,0.174113,9.414956,6.692883,13.244128,0.903984,0.870010,0.929796,12.878432,5.953249e-38,***
time_budget_z,-0.327912,-0.486948,-0.168876,0.081142,0.720426,0.614499,0.844613,0.418749,0.380613,0.457881,-4.041205,5.317721e-05,***
dwell_mirror_z,-0.321945,-0.440772,-0.203119,0.060627,0.724738,0.643540,0.816181,0.420202,0.391557,0.449394,-5.310285,1.094536e-07,***
RT_z,-1.909833,-2.028957,-1.790709,0.060779,0.148105,0.131473,0.166842,0.129000,0.116196,0.142986,-31.422817,9.873984e-217,***
d_z:dwell_mirror_z,-0.056345,-0.130568,0.017878,0.037869,0.945213,0.877597,1.018039,0.485917,0.467404,0.504469,-1.487875,1.367840e-01,
tta_z:dwell_mirror_z,0.135669,0.019393,0.251945,0.059325,1.145303,1.019583,1.286525,0.533865,0.504848,0.562655,2.286861,2.220393e-02,*
time_budget_z:dwell_mirror_z,0.208178,0.113350,0.303006,0.048382,1.231433,1.120024,1.353923,0.551857,0.528307,0.575177,4.302760,1.686832e-05,***


In [19]:
coefs = model_decision_dwell_RT.coefs.loc[:, ["Estimate", "SE", "Z-stat", "P-val"]]
coefs["P-val"] = coefs["P-val"].apply(p_formatted)
styler = coefs.rename(columns=column_names, index=var_names).style.format(precision=2)

with open(os.path.join(output_path, "tab_decision_dwell_RT.tex"), 'w') as writer:
     writer.write(styler.to_latex(
         column_format="rrrrr", position="!ht", position_float="centering",
         hrules=True, label="tab:decision_dwell_RT", caption="Standardized coefficients of the mixed-effects logistic regression describing the decision outcome as a function of kinematic variables, response time, and relative dwell time. Random slopes of TTA to the overtaking vehicle and the time budget provided by the merging lane were included per participant: \\texttt{decision $\sim$ 1 + RT + distance + (TTA * time budget * \\% dwell time mirror) + (1 + TTA + time budget) | participant}."
     )
)